In [1]:
!pip install datasets

from datasets import load_dataset

dataset = load_dataset("SKNahin/bengali-transliteration-data")
print(dataset)


split_ratio = 0.1
train_val_split = dataset["train"].train_test_split(test_size=split_ratio, seed=42)

train_dataset = train_val_split["train"]
val_dataset = train_val_split["test"]

print(f"{len(train_dataset)}")
print(f"{len(val_dataset)}")

print(train_dataset[0])
print(val_dataset[0])


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/300 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/333k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5006 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['bn', 'rm'],
        num_rows: 5006
    })
})
Number of training samples: 4505
Number of validation samples: 501

Sample training data: {'bn': 'আপনার এফবি আইডি নেম বিশাল আর এইখানে মামুন কেন ?', 'rm': 'Apnar fb id name Bishal ar Ekhane mamun keno ?'}

Sample validation data: {'bn': 'ভালো করে ট্রাই করেন পাবেন..', 'rm': 'valo kore trai koren paben..'}


In [3]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "google/mt5-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

print(f"Loaded model: {model_name}")

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Loaded model: google/mt5-small


Reason for using google/mt-5-small:
Multilingual Strength: mT5 is pre-trained on multiple languages, including Bengali, making it ideal for translating Banglish (Romanized Bengali) to proper Bengali script.

Good for Low-Resource Languages: Bengali is a low-resource language, and mT5 is trained on diverse data, allowing it to handle languages with less available training data effectively.

Efficient and Lightweight: The small version of mT5 is more resource-efficient, making it faster to train with less memory usage, which is ideal for platforms like Google Colab.

Sequence-to-Sequence Tasks: mT5 works good in sequence-to-sequence tasks like translation and transliteration, which fits perfectly for your task of converting Banglish to Bengali.

In [4]:
!pip install transformers sentencepiece

from transformers import AutoTokenizer
import re


bangla_tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")  # Pre-trained Bangla tokenizer


def tokenize_banglish(text):
    return list(text)

def clean_data(example):
    banglish = example["rm"]
    bangla = example["bn"]

    if len(banglish) < 3 or len(banglish) > 100 or len(bangla) < 3 or len(bangla) > 100:
        return False


    if not re.match("^[a-zA-Z0-9 /:,.?!]+$", banglish):
        return False

    return True

filtered_train = train_dataset.filter(clean_data)
filtered_val = val_dataset.filter(clean_data)


def tokenize_function(example):
    # Tokenize Banglish input
    banglish_tokens = tokenizer(
        example["rm"], truncation=True, padding="max_length", max_length=128
    )

    # Tokenize Bangla target
    bangla_tokens = tokenizer(
        example["bn"], truncation=True, padding="max_length", max_length=128
    )

    return {
        "input_ids": banglish_tokens["input_ids"],
        "attention_mask": banglish_tokens["attention_mask"],
        "labels": bangla_tokens["input_ids"]
    }


tokenized_train = filtered_train.map(tokenize_function, batched=True)
tokenized_val = filtered_val.map(tokenize_function, batched=True)





Map:   0%|          | 0/3817 [00:00<?, ? examples/s]

Map:   0%|          | 0/412 [00:00<?, ? examples/s]

Pre-trained indic-bert Tokenizer:

I selected the pre-trained ai4bharat/indic-bert tokenizer to tokenize the Bengali text (Bangla) in the dataset. IndicBERT is a multilingual BERT-based model pre-trained on 12 Indian languages, including Bengali. This tokenizer is optimized for languages with a rich morphology, such as Bengali, and can handle the script-specific nuances efficiently.
Reasoning for Use: Given that the target language is Bengali, using a tokenizer that has been pre-trained on a similar language ensures that the model has prior knowledge of the language structure and vocabulary. This helps the model generate better translations during fine-tuning, as the tokenizer will already understand the subword units of Bengali.

Banglish Tokenizer (Custom tokenize_banglish Function):

For Banglish (Bengali written in English letters), I implemented a custom tokenization function: tokenize_banglish(), which treats the input text as a list of individual characters. This approach is effective because Banglish text is typically written in the English alphabet with Bengali phonetics, and using a character-level tokenizer helps to capture the word structure even in informal or phonetic representations.
Reasoning for Use: Since Banglish can be highly variable and phonetically inconsistent, treating the text as a sequence of individual characters helps in breaking down and processing the input text into manageable tokens. This allows the model to learn mappings between Banglish and Bengali script at a granular level. By avoiding standard word-level tokenization, we reduce the impact of inconsistencies in spelling and provide the model with greater flexibility in handling different variations of Banglish.

In [5]:
tokenized_train = tokenized_train.remove_columns(["rm", "bn"])
tokenized_val = tokenized_val.remove_columns(["rm", "bn"])

In [6]:
!pip install transformers datasets evaluate torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00


In [7]:
from datasets import DatasetDict
from transformers import DataCollatorForSeq2Seq


dataset_dict = DatasetDict({
    "train": tokenized_train,
    "validation": tokenized_val,
})

# Data collator for padding
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)


In [14]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=50,
    report_to="none",
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Parameter justifications:
Learning Rate (3e-5): A small learning rate like 3e-5 is commonly used in fine-tuning transformer models. It helps prevent overshooting the optimal solution and ensures stable training, especially for large models like mT5.

Batch Size (8): A batch size of 8 strikes a balance between memory usage and model performance. It's small enough to fit on limited GPU memory (like in Google Colab), but large enough to provide meaningful gradient updates. Initially I tried to use batch size=20, but got memory error and then tried using 8 and got succedeed training.

Number of Epochs (5): Five epochs are chosen to give the model enough iterations to learn from the data without overfitting. Since the dataset is relatively small, this number of epochs helps the model converge without excessive training time.

In [15]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


<ipython-input-15-171c91793687>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [16]:

trainer.train()


trainer.save_model("./fine_tuned_mt5")


Epoch,Training Loss,Validation Loss
1,18.255800,11.225633
2,6.683800,4.003407
3,2.907000,1.811765
4,1.889200,1.221051
5,1.747200,1.107054
